# Data Wrangling in Pandas

In [1]:
import pandas as pd
import numpy as np

## Merging

In [2]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],'data1': range(7)})
df2 = pd.DataFrame({'key': ['a', 'b', 'd'],'data2': range(3)})
print df1
print df2

   data1 key
0      0   b
1      1   b
2      2   a
3      3   c
4      4   a
5      5   a
6      6   b
   data2 key
0      0   a
1      1   b
2      2   d


In [3]:
# Here is a many to one merge.  The join field is implicit, based on what column name in common between the two dataframes.
pd.merge(df1,df2)

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


In [4]:
# Here is the same merge, but making the join field explicit.
pd.merge(df1,df2, on='key')

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


In [5]:
#what if there are more than one value of key in both dataframes? 
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],'data1': range(7)})
df3 = pd.DataFrame({'key': ['a', 'b', 'b', 'd'],'data2': range(4)})
print df1
print df3
pd.merge(df1,df3, on='key')
#This produces a cartesian product of the number of occurrences of each key value in both dataframes:
# (b shows up 3 times in df1 and 2 times in df3, so we get 6 occurrences in the result of the merge)

   data1 key
0      0   b
1      1   b
2      2   a
3      3   c
4      4   a
5      5   a
6      6   b
   data2 key
0      0   a
1      1   b
2      2   b
3      3   d


,data1,key,data2
0,0,b,1
1,0,b,2
2,1,b,1
3,1,b,2
4,6,b,1
5,6,b,2
6,2,a,0
7,4,a,0
8,5,a,0


In [6]:
# missing string values are NaN in pandas too
x = pd.DataFrame({'a':['abc'], 'b':['def']})
y = pd.DataFrame({'a':['xyz']})
pd.merge(x, y, on='a', how='outer')

,a,b
0,abc,def
1,xyz,NaN


In [7]:
# There are several types of joins: left, right, inner, and outer. Let's compare them.
# How does a 'left' join compare to our initial join?
# left makes sure rows are there if they are in the left df
pd.merge(df1,df3, on='key', how='left')

,data1,key,data2
0,0,b,1
1,0,b,2
2,1,b,1
3,1,b,2
4,2,a,0
5,3,c,NaN
6,4,a,0
7,5,a,0
8,6,b,1
9,6,b,2


In [8]:
#How does a 'right' join compare?
# right makes sure rows are there if they were in the right df
pd.merge(df1,df3, on='key', how='right')

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,0,b,2
4,1,b,2
5,6,b,2
6,2,a,0
7,4,a,0
8,5,a,0
9,NaN,d,3


In [9]:
#How does an 'inner' join compare?
pd.merge(df1,df3, on='key', how='inner')
# seems to be the default argument...

,data1,key,data2
0,0,b,1
1,0,b,2
2,1,b,1
3,1,b,2
4,6,b,1
5,6,b,2
6,2,a,0
7,4,a,0
8,5,a,0


In [10]:
#How does an 'outer' join compare?
pd.merge(df1,df3, on='key', how='outer')

,data1,key,data2
0,0,b,1
1,0,b,2
2,1,b,1
3,1,b,2
4,6,b,1
5,6,b,2
6,2,a,0
7,4,a,0
8,5,a,0
9,3,c,NaN


In [11]:
#What if the join fields have different names?
df4 = pd.DataFrame({'key_1': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],'data1': range(7)})
df5 = pd.DataFrame({'key_2': ['a', 'b', 'b', 'd'],'data2': range(4)})
pd.merge(df4,df5, left_on='key_1', right_on='key_2')


,data1,key_1,data2,key_2
0,0,b,1,b
1,0,b,2,b
2,1,b,1,b
3,1,b,2,b
4,6,b,1,b
5,6,b,2,b
6,2,a,0,a
7,4,a,0,a
8,5,a,0,a


In [12]:
# Here is an example that uses a combination of a data column and an index to merge two dataframes:
df4 = pd.DataFrame({'key_1': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],'data1': range(7)})
df5 = pd.DataFrame({'data2': [4,6,8,10]}, index=['a','b','c','d'])
pd.merge(df4,df5, left_on='key_1', right_index=True)

,data1,key_1,data2
0,0,b,6
1,1,b,6
6,6,b,6
2,2,a,4
4,4,a,4
5,5,a,4
3,3,c,8


## Concatenating

In [13]:
# Concatenating can append rows, or columns, depending on which axis you use. Default is 0
s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 6], index=['f', 'g'])
pd.concat([s1, s2, s3])
# Since we are concatenating series on axis 0, this creates a longer series, appending each of the three series

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [14]:
# What if we concatenate on axis 1?
pd.concat([s1, s2, s3], axis=1)

,0,1,2
a,0,NaN,NaN
b,1,NaN,NaN
c,NaN,2,NaN
d,NaN,3,NaN
e,NaN,4,NaN
f,NaN,NaN,5
g,NaN,NaN,6


In [15]:
# Outer join is the default:
pd.concat([s1, s2, s3], axis=1, join='outer')

,0,1,2
a,0,NaN,NaN
b,1,NaN,NaN
c,NaN,2,NaN
d,NaN,3,NaN
e,NaN,4,NaN
f,NaN,NaN,5
g,NaN,NaN,6


In [16]:
# What would an inner join produce?
pd.concat([s1, s2, s3], axis=1, join='inner')

,0,1,2


In [17]:
# We need some overlapping values to have the inner join produe non-empty results
s4 = pd.Series([4, 5, 6], index=['c', 'd', 'e'])
s5 = pd.Series([1, 2, 3], index=['d', 'e', 'f'])
s6 = pd.Series([7, 8, 9, 10], index=['d', 'e', 'f', 'g'])
pd.concat([s4, s5, s6], axis=1, join='outer')

,0,1,2
c,4,NaN,NaN
d,5,1,7
e,6,2,8
f,NaN,3,9
g,NaN,NaN,10


In [18]:
# Here is the inner join 
pd.concat([s4, s5, s6], axis=1, join='inner')
# Note that it contains only entries that overlap in all three series.

,0,1,2
d,5,1,7
e,6,2,8


## Reshaping with Hierarchical Indexing

In [19]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                 index=pd.Index(['Ohio', 'Colorado'], name='state'),
                 columns=pd.Index(['one', 'two', 'three'], name='number'))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [20]:
# Stack pivots the columns into rows, producing a Series with a hierarchical index:
result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

In [21]:
# Unstack reverses this process:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


See also the related pivot method

## Data Transformations

In [22]:
# Start with a dataframe containing some duplicate values
data = pd.DataFrame({'k1': ['one'] * 3 + ['two'] * 4,'k2': [1, 1, 2, 3, 3, 4, 99]})
data

,k1,k2
0,one,1
1,one,1
2,one,2
3,two,3
4,two,3
5,two,4
6,two,99


In [23]:
# How to see which rows contain duplicate values
data.duplicated()

0    False
1     True
2    False
3    False
4     True
5    False
6    False
dtype: bool

In [24]:
# How to remove duplicate values
data.drop_duplicates()

,k1,k2
0,one,1
2,one,2
3,two,3
5,two,4
6,two,99


In [25]:
#If 99 is a code for missing data, we could replace any such values with NaNs
data.replace(99,np.nan)

,k1,k2
0,one,1
1,one,1
2,one,2
3,two,3
4,two,3
5,two,4
6,two,NaN


In [26]:
# Let's look at how to create categories of data using ranges to bin the data using cut
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]
bins = [18, 25, 35, 60, 100]
cats = pd.cut(ages, bins)
type(cats)

pandas.core.categorical.Categorical

In [27]:
cats.categories

Index([u'(18, 25]', u'(25, 35]', u'(35, 60]', u'(60, 100]'], dtype='object')

In [28]:
cats.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [29]:
pd.value_counts(cats)

(18, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
dtype: int64

In [30]:
# Consistent with mathematical notation for intervals, a parenthesis means that the side is open while the 
#square bracket means it is closed (inclusive). Which side is closed can be changed by passing right=False:
cats = pd.cut(ages, bins, right=False)
print ages
print pd.value_counts(cats)

[20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]
[25, 35)     4
[18, 25)     4
[35, 60)     3
[60, 100)    1
dtype: int64


### Removing Outliers

In [31]:
# Start by creating a dataframe with 4 columns of 1,000 random numbers
# We'll use a fixed seed for the random number generator to get repeatable results
np.random.seed(12345)
data = pd.DataFrame(np.random.randn(1000, 4))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.067684,0.067924,0.025598,-0.002298
std,0.998035,0.992106,1.006835,0.996794
min,-3.428254,-3.548824,-3.184377,-3.745356
25%,-0.774890,-0.591841,-0.641675,-0.644144
50%,-0.116401,0.101143,0.002073,-0.013611
75%,0.616366,0.780282,0.680391,0.654328
max,3.366626,2.653656,3.260383,3.927528


In [32]:
# This identifies any values in column 3 with absolute values > 3
col = data[3]
col[np.abs(col) > 3]

97     3.927528
305   -3.399312
400   -3.745356
Name: 3, dtype: float64

In [33]:
# This identifies all the rows with any column containing absolute values > 3
data[(np.abs(data) > 3).any(axis=1)]

,0,1,2,3
5,-0.539741,0.476985,3.248944,-1.021228
97,-0.774363,0.552936,0.106061,3.927528
102,-0.655054,-0.565230,3.176873,0.959533
305,-2.315555,0.457246,-0.025907,-3.399312
324,0.050188,1.951312,3.260383,0.963301
400,0.146326,0.508391,-0.196713,-3.745356
499,-0.293333,-0.242459,-3.056990,1.918403
523,-3.428254,-0.296336,-0.439938,-0.867165
586,0.275144,1.179227,-3.184377,1.369891
808,-0.362528,-3.548824,1.553205,-2.186301


In [34]:
# Now we can cap the values at -3 to 3 using this:
data[np.abs(data) > 3] = np.sign(data) * 3
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.067623,0.068473,0.025153,-0.002081
std,0.995485,0.990253,1.003977,0.989736
min,-3.000000,-3.000000,-3.000000,-3.000000
25%,-0.774890,-0.591841,-0.641675,-0.644144
50%,-0.116401,0.101143,0.002073,-0.013611
75%,0.616366,0.780282,0.680391,0.654328
max,3.000000,2.653656,3.000000,3.000000


### Computing Dummy Variables

In [35]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],'data1': range(6)})
df

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,b


In [36]:
# This generates dummy variables for each value of key
# Dummy variables are useful in statistical modeling, to have 0/1 indicator
# variables for the presence of some condition
pd.get_dummies(df['key'])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [37]:
# This generates dummy variables for each value of key and appends these to the dataframe
dummies = pd.get_dummies(df['key'], prefix='key')
df_with_dummy = df[['data1']].join(dummies)
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


## Vectorized String Methods
We have covered most of these arguments for vectorized string functions already. 
This is a handy reference to the available arguments.

 Argument | Description
 --- | ---
 count | Return the number of non-overlapping occurrences of substring in the string.
 endswith, startswith | Returns True if string ends with suffix (starts with prefix).
 join | Use string as delimiter for concatenating a sequence of other strings.
 index | Return position of first character in substring if found in the string. Raises ValueEr ror if not found.
find | Return position of first character of first occurrence of substring in the string. Like index, but returns -1 if not found.
rfind | Return position of first character of last occurrence of substring in the string. Returns -1 if not found.
 replace | Replace occurrences of string with another string.
 strip, rstrip, lstrip | Trim whitespace, including newlines; equivalent to x.strip() (and rstrip,lstrip, respectively) for each element.
 split | Break string into list of substrings using passed delimiter.
 lower, upper | Convert alphabet characters to lowercase or uppercase, respectively.
 ljust, rjust | Left justify or right justify, respectively. Pad opposite side of string with spaces (or some other fill character) to return a string with a minimum width.

# Reviewing our earlier application of Data Wrangling to Craigslist Data

In [38]:
# import libraries and read in the csv file
import re as re, pandas as pd, numpy as np, requests, json
df = pd.read_csv('bay.csv')

# clean price and neighborhood
df.price = df.price.str.strip('$').astype('float64')
df.neighborhood = df.neighborhood.str.strip().str.strip('(').str.strip(')')

# break out the date into month day year columns
df['month'] = df['date'].str.split().str[0]
df['day'] = df['date'].str.split().str[1].astype('int32')
df['year'] = df['date'].str.split().str[2].astype('int32')
del df['date']

# clean bedrooms using regular expressions
# see https://docs.python.org/2/howto/regex.html for a tutorial - it is a moderately advanced topic
for i in df.bedrooms.index:
    r = re.search('(?<=\/ )(.*)(?=br)', df.loc[i,'bedrooms'])
    df.loc[i,'bedrooms'] = r.group(0) if r else np.nan
df.bedrooms = df.bedrooms.astype('float64')


# clean up the sqft using an iterator over the rows in the DataFrame
for i in df.sqft.index:
    if('ft' in df.sqft[i]):
        end = df.sqft[i].find('ft')
        begin = df.sqft[i].find('- ') + 2
        if(begin > end):
            begin = df.sqft[i].find('/ ') + 2
        df.loc[i,'sqft'] = df.sqft[i][begin:end]
    else:
        df.loc[i,'sqft'] = np.nan
df.sqft = df.sqft.astype('float64')


df.head(2)

,neighborhood,title,price,bedrooms,pid,longitude,subregion,link,latitude,sqft,month,day,year
0,bayview,Take A TOUR ON OUR ONE FURNISHED BEDROOM TODAY,950,1,4076905111,-122.396965,SF,/sfc/apa/4076905111.html,37.761216,NaN,Sep,18,2013
1,bayview,Only walking distance to major shopping centers.,950,1,4076901755,-122.396793,SF,/sfc/apa/4076901755.html,37.761080,NaN,Sep,18,2013


## Let's do some wrangling on this dataset:
1. Find outliers in rent, say below 200 or above 10,000
2. Create a dataset that removes the outliers


In [39]:
df.describe()

,price,bedrooms,pid,longitude,latitude,sqft,day,year
count,4908.000000,4514.000000,5.000000e+03,3143.000000,3143.000000,3178.000000,5000.000000,5000
mean,2656.999389,2.068675,4.068059e+09,-122.264948,37.757411,1173.613593,17.523800,2013
std,1915.147477,1.012739,1.344453e+07,0.278825,0.364646,751.552623,0.766258,0
min,1.000000,1.000000,4.008227e+09,-123.799100,36.813820,1.000000,14.000000,2013
25%,1695.000000,1.000000,4.065685e+09,-122.442365,37.469365,747.250000,17.000000,2013
50%,2208.500000,2.000000,4.074290e+09,-122.283714,37.760858,1000.000000,18.000000,2013
75%,2995.000000,3.000000,4.075949e+09,-122.045047,37.900832,1350.000000,18.000000,2013
max,35000.000000,8.000000,4.076908e+09,-120.034132,41.456848,12700.000000,18.000000,2013


In [40]:
df[(df['price'] < 200)].describe()

,price,bedrooms,pid,longitude,latitude,sqft,day,year
count,4.0,2.000000,4.000000e+00,2.000000,2.000000,2.000000,4,4
mean,49.5,1.500000,4.075769e+09,-122.760199,38.689706,3650.000000,18,2013
std,97.0,0.707107,2.268995e+05,0.024750,0.356939,2333.452378,0,0
min,1.0,1.000000,4.075510e+09,-122.777700,38.437312,2000.000000,18,2013
25%,1.0,1.250000,4.075657e+09,-122.768950,38.563509,2825.000000,18,2013
50%,1.0,1.500000,4.075755e+09,-122.760199,38.689706,3650.000000,18,2013
75%,49.5,1.750000,4.075867e+09,-122.751448,38.815903,4475.000000,18,2013
max,195.0,2.000000,4.076056e+09,-122.742698,38.942100,5300.000000,18,2013


In [41]:
df[(df['price'] > 10000)].describe()

,price,bedrooms,pid,longitude,latitude,sqft,day,year
count,51.00000,51.000000,5.100000e+01,30.000000,30.000000,32.000000,51.000000,51
mean,14664.00000,4.470588,4.061403e+09,-122.406954,37.770468,4143.718750,17.490196,2013
std,5414.82683,1.301583,1.713079e+07,0.143792,0.146150,2096.487327,0.784157,0
min,10100.00000,2.000000,4.017078e+09,-122.555237,37.238734,1418.000000,14.000000,2013
25%,11000.00000,4.000000,4.057289e+09,-122.469483,37.786283,2881.000000,17.000000,2013
50%,12995.00000,4.000000,4.068016e+09,-122.441534,37.794419,3540.500000,18.000000,2013
75%,16000.00000,5.000000,4.075096e+09,-122.419420,37.803752,5268.750000,18.000000,2013
max,35000.00000,8.000000,4.076616e+09,-121.911587,37.959696,11685.000000,18.000000,2013


In [42]:
filtered = df[(df['price'] < 10000) & (df['price'] > 200)]
filtered.describe()

,price,bedrooms,pid,longitude,latitude,sqft,day,year
count,4851.000000,4460.000000,4.851000e+03,3094.000000,3094.000000,3138.000000,4851.000000,4851
mean,2529.888889,2.041256,4.068082e+09,-122.262421,37.755882,1142.331421,17.526696,2013
std,1359.640216,0.975641,1.342302e+07,0.278971,0.365073,656.783855,0.761617,0
min,250.000000,1.000000,4.008227e+09,-123.799100,36.813820,1.000000,14.000000,2013
25%,1695.000000,1.000000,4.065695e+09,-122.440282,37.466532,745.000000,17.000000,2013
50%,2200.000000,2.000000,4.074304e+09,-122.278227,37.759540,995.000000,18.000000,2013
75%,2950.000000,3.000000,4.075956e+09,-122.042879,37.901224,1333.750000,18.000000,2013
max,9999.000000,8.000000,4.076908e+09,-120.034132,41.456848,12700.000000,18.000000,2013


## OK, now on your own: 
1. try filtering out records with more than 4 bedrooms
2. create dummy variables for each bedroom count (e.g. bed_1 would have 1 for rows with 1 bedroom, 0 for others)